# Instrucciones

En este Notebook va a implementar los conceptos aprendidos en clase. Para ello implementara un __autoencoder__. Asimismo usara las librerias `UMAP` y `scikit-learn`. Antes de comenzar, proceda a instalar las librerias necesarias con lo siguientes comandos:

In [ ]:
!pip install umap-learn

In [ ]:
!pip install umap-learn[plot]

In [ ]:
!pip install -U scikit-learn==1.0.2

Ahora proceda a descargar los datos y funciones adicionales con la siguiente linea:

In [ ]:
!curl -OL https://raw.githubusercontent.com/aguilarls/practicas/main/Algebra-computarizada/files.tar.xz && tar -xf ./files.tar.xz

# 1. Aplicaciones de algebra lineal en datasets

In [ ]:
from helpers import *

## 2. Dataset

Para esta aplicacion, vamos a usar un subset del dataset __MNIST__. Este dataset contiene imagenes de numeros del 0 al 9 escritos a mano. Las dimensiones de las imagenes son `28 * 28` pixeles. Usaremos la siguiente funcion para cargar los datos.

In [ ]:
data, targets = get_data('dataset')

<div class="alert-info">
1. Pregunta: Cuantos datos y caracteristicas contiene el data set?.
</div>

Ahora vamos a visualizar una parte del dataset. Para ello ejecutaremos el siguiente comando:

In [ ]:
plot_data(data)

## 3. Preprocesamiento

En esta seccion vamos a realizar el preprocesamiento de los datos. Esta es una etapa fundamental que se debe de realizar antes de poder aplicar redes neuronales. Existen diversas opciones de preprocesamiento, se debe de elegir la que mas se adecua a los datos y modelo a usar. En este caso vamos a __escalar__ (transformar) los datos aplicando la siguiente ecuacion:

$$
\large X_{s} = X_{\sigma} * (a - b) + b
$$
Donde:
* $ \large X_{\sigma} = \frac{X - X_{min}}{X_{max} - X_{min}}$
* $a$ representa el limite inferior.
* $b$ representa el limite mayor.

Ya que vamos a trabajar con __redes neuronales__, vamos a usar `0` y `1` para los intervalos $(a,b)$, esto ayudara a la convergencia de nuestra red. Asimismo, ya que estamos trabajando con imagenes, los valores `0` y `1` normalizaran la intensidad de los pixeles. Para la implementacion de la ecuacion vamos a usar la `clase` [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn-preprocessing-minmaxscaler) from `scikit-learn`, donde especificaremos $(a, b)$ con el argumento `feature_range=(0, 1)`

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = ...

Ahora procederemos a preprocesar los datos usando el comando `fit_transform`. Este comando se divide en dos acciones. Primero, `fit` ajusta la ecuacion a los datos, en este caso calcula $X_{\sigma}$. Luego `transform` transforma los datos aplicando la ecuacion. Este proceso nos devuelve los datos __preprocesados__ en la variable `X_train_norm`.

In [ ]:
X_train_norm = ...

Notemos que los valores de `X_train_norm` estan transformados de acuerdo al rango $(a,b)$ previamente definido. Para comprobarlo, imprimamos los valores maximos y minimos con las funciones `max` y `min`:

In [ ]:
print('Max value {}'.format(X_train_norm.max()))
print('Min value {}'.format(X_train_norm.min()))

In [ ]:
print(X_train_norm)

---

## 4. Autoencoders

En esta sección, vamos a aplicar una red de autocodificador __(autoencoder)__. Primero comenzaremos definiendo que es un __autoencoder__. Para ello, definamos $h_{\Theta}$ como el __autoencoder__, donde $\Theta$ representa los parametros del modelo. En este contexto, para una entrada $X$, el objetivo del __autoencoder__ es:

$$
h_{\Theta}(X) = d(e(X)) = \hat X
$$

Donde $e(X)$ y $d(e(X))$ estan compuestos por capas y son conocidos como encodificador (__encoder__) y decodificador (__decoder__) respectivamente. La funcion de $e(X)$ consiste en reducir las dimensiones de $X$ hasta llegar a la maxima compresion en la capa de codigo (__code__), donde se obtiene una __representacion encodificada__ $E$. Esta representacion $E$ es usada por $d(E)$ para reconstruir la entrada original $X$ tal que $d(E) = \hat X$.

Para ilustrar mejor este proceso, vamos a definir un __autoencoder__ para nuestro dataset. Para ello, observamos la siguiente figura:

<div align="center">
<img src="https://raw.githubusercontent.com/aguilarls/practicas/main/Algebra-computarizada/images/autoencoder.png" />
</div>

Como podemos apreciar, tenemos de entrada $X$ una imagen compuesta por `28 * 28` pixels. Comenzemos a definir la arquitectura:

__1. Encoder:__ Para poder usar esta representacion, vamos a convertir nuestros datos de matrix en un vector: `28 * 28 = 784`. De esta manera, nuestra capa de entrada cuenta con `784` unidades. A continuacion definiremos una capa oculta. Para el numero de neuronas consideraremos la reduccion de `8` pixels de la imagen orginal $X$, esto se traduce en: `20 * 20 = 400` unidades. Ahora procederemos a definir el __code__.

__2. Code:__ A continuacion vamos a definir las unidades del __codigo__, para ello vamos a reducir `10` pixeles, obteniendo: `10 * 10 = 100` unidades. Como siguiente paso procederemos con el __decoder__.

__3. Decoder:__ Para el proceso de construccion del __decoder__ vamos a usar el mismo numero de capas y neuronas del __encoder__ pero de forma invertida. De ese modo, la primera capa tendria: `20 * 20 = 400` unidades. Finalmente, la capa de salida contendria: `28 * 28 = 784` unidades.

Es importante notar que, tanto la entrada del __decoder__ como la salida del __encoder__ deben de coincidir en el numero de __neuronas__. Finalmente, como funcion de activacion vamos a usar relu:
$$
x = max(0, x)
$$

Durante la implementacion de su `autoencoder` puede usar el numero de `neuronas` y `capas` que considere necesario con la siguiente __condiciones:__
1. El numero de neuronas de su capa de entrada (`input_size`) debe de coincidir con el numero de salida (`output_size`).
2. Con el fin de visualizar su __code__, debe utilizar las mismas dimenciones para al `ancho` y `alto`, por ejemplo consideremos `7` pixels de `ancho` y `largo`, para este caso tendria un total de `7 * 7 = 49` neuronas en su __code__.
3. Como funcion de activacion debe de usar `relu`.

Como sugerencia se le recomienda realiazar las mismas reducciones al `largo` y `ancho` de la imagen en su `encoder`. Por ejemplo, para un `encoder` con 3 capas: `28 * 28 = 784` -> `25 * 25 = 625` -> `23 * 23 = 529`. Recuerde usar `variables` para representar el numero de neuronas en sus capas.

__Note__ que la funcion `eval_autoencoder` evaluara si su arquitectura esta correcta, se le recomienda no proceder con el ejercicio y/o realizar las correciones necesarias si hubiese algun error.

A continuacion, definira su modelo, recuerde que __debe__ agregar capas, la cantidad dependera de usted. Las siguientes `variables` se le han proporcionado:
* __input_size__: Dimensiones de entrada.
* __n_code__: Dimensiones de su code.
* __output_size__: Dimensiones de salida.
* __activation_f__: Funcion de activacion.

In [ ]:
# autoencoder model
input_size = ...   # 28 * 28

n_code = ...    

output_size = ... # 28 * 28
activation_f = ...

Debido a que usaremos tuplas, agruparemos las capas en el orden adecuado. Para ello usaremos la variable `hidden_layers`.

In [ ]:
hidden_layers = (...)

Si queremos construir un __autoencoder__ con más capas, podemos agregarlas usando variables, por ejemplo: `hidden_layers = (input_size, encoder_size_1, encoder_size_2, ..., n_encoded, decoder_size_1, decoder_size_2, ..., output_size)`. Tenga en cuenta que `input_size` debe tener las mismas dimensiones que `otput_size`

Para la construccion del __autoencoder__ usaremos la clase [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn-neural-network-mlpregressor) de `scikit-learn`.

In [ ]:
from sklearn.neural_network import MLPRegressor

A continuacion definira los `epochs` y el ratio de aprendizaje `lr`:

In [ ]:
# training hyper-params
epochs = ...
lr = ...

La clase `MLPRegressor` le permite crear una red neuronal. Para este caso, debe de completar los siguientes argumentos:
1. `hidden_layer_sizes`: Representa el numero de capas y neuronas de su red definidas por la variable: `hidden_layers`.
2. `max_iter`: Denota el numero maximo de `epochs`.
3. `learning_rate_init`: Especifica el ratio de aprendizaje `lr`.
4. `activation`: Representa la funcion de activacion `activation_f`.

In [ ]:
auto_encoder = MLPRegressor(hidden_layer_sizes = ...,
                            max_iter = ..., 
                            learning_rate_init = ...,
                            activation = ...,
                            verbose = 1)

Antes de proceder, nos aseguraremos de que nuestra arquitectura este correcta:

In [ ]:
eval_autoencoder(auto_encoder)

Para corregir los errores, proceda a modificar y/o ejecuar la __arquitectura__ definida en:
```python
# autoencoder model
input_size = ...   # 28 * 28

n_code = ...    

output_size = ... # 28 * 28
activation_f = ...
```
Para aplicar los cambios ejecute las celdas con las variables `hidden_layers` y `auto_encoder`.

<div class="alert-info">
2. Pregunta: Describa en detalle su arquitectura. Cuantas unidades decicio usar en el code y por que?
</div>

Para el entrenamiento, usaremos los datos __preprocesados__ `X_train_norm` como $(X,y)$, ya que deseamos aprender a reconstruir los datos originales $X$, la entrada y salida son las mismas.

In [ ]:
auto_encoder.fit(...)

Podemos observar como ha ido evoluacionando el loss con el siguiente grafico.

In [ ]:
plot_loss(auto_encoder)

<div class="alert-info">
3. Pregunta: Cual fue el loss minimo que alcanzo su autoencodificador, y en cuantas iteraciones? 
</div>

Una vez entrenado el autoencodificador, vamos a realizar algunas predicciones. Ejecute la siguiente celda las veces necesarias para generar predicciones. Note que de entrada se encuentra una imagen en la izquierda (__Input image__), la imagen del centro contiene el __Code representation__, es decir la representacion comprimida de su modelo. Finalmente, el decodificador usa el __code__ para reconstruir nuevamente la imagen (__reconstructed image__).

In [ ]:
plot_prediction(X_train_norm, auto_encoder, targets)

En la imagen, tenemos como entrada un $X$ original que representa un numero. El grafico del centro muestra la representacion $E$ obtenida por el __codigo__. Finalmente la imagen de la derecha muestra $X^{'}$.

---

## 5. UMAP como alternativa a PCA

Tanto `UMAP` como `PCA` se pueden utilizar para la reducción dimensional. Por ejemplo, sea $X^{(m,n)}$ una matriz de entrada donde $m$ y $n$ representan el total de filas y columnas respectivamente. Asimismo, sea $z$ una nueva dimensión tal que $z < n$. Aplicando `UMAP` o `PCA` sobre la estructura original $X^{(m,n)}$ obtendremos una nueva matriz $X^{(m,z)}$. Dado que tanto `UMAP` como `PCA` guardan información sobre la estructura original $X^{(m,n)}$, es posible realizar la operación __inversa (reconstruccion)__, es decir: $X^{(m,n)} \rightarrow X^{(m,z)}$.

Por supuesto, ambos algoritmos usan __diferentes métodos__ para transformar e invertir tales transformaciones, siendo `UMAP` el mas complejo. Otro caso de uso es la visualización. Por ejemplo: $X^{(m,100)} \rightarrow X^{(m,2|3)}$. En esta sección vamos a transformar y visualizar los datos generados por nuestro __encoder__ $E$ usando ambos algoritmos tal que: $E^{(m,n)} \rightarrow E^{(m,2)}$.

<div class="alert-info">
4. Pregunta: Para un dataset $X^{(m,n)}$, el proceso de reduccion a una nueva dimension $z$ debe garantizar que:
    <li>$z>=n$.</li>
    <li>$z>n$.</li>
    <li>$n=z$.</li>
    <li>$n<z$.</li>
    <li>$z<n$.</li>
</div>

In [ ]:
from umap import UMAP
from sklearn.decomposition import PCA

Primero obtendremos $E$, para ello usaremos el __encoder__ que nos devolvera los datos en la variable `encoded_data`.

In [ ]:
encoded_data = encoder(X_train_norm, auto_encoder)

Como podemos comprobar, $E$ contiene la representacion comprimida de todos los datos:

In [ ]:
encoded_data.shape

Para este ejercicio vamos a reducir las dimensiones a 2: $E^{(m,n)} \rightarrow E^{(m,2)}$. Para ello, definiremos una variable llamada `new_dims` con las nuevas dimensiones reducidas.

In [ ]:
new_dims = ...

Tanto `UMAP` como `PCA` tienen un argumento llado `n_components`, el cual indica a cuantas dimensiones se va a convertir. Para nuestro caso definiremos: `n_components = new_dims`. Asimismo, el argumento `random_state` nos permite obtener resultados reproducibles, para este caso, asignaremos el valor de `5`.

In [ ]:
umap = UMAP(n_components = ..., random_state = ...)
pca = PCA(n_components = ..., random_state = ...)

Ahora procederemos a entrenar y transformar $E$. Para ello proporcionaremos como entrada la variable `encoded_data` a `UMAP` y `PCA`. Asimismo definiremos dos variables: `dims_umap` y `dims_pca` para guardar $E$ con las nuevas dimensiones. 

In [ ]:
dims_umap = umap.fit_transform(...)
dims_pca = pca.fit_transform(...)

Inspeccionemos las nuevas dimensiones:

In [ ]:
print('Original encoded data E: {}'.format(encoded_data.shape))
print('UMAP reduced data E: {}'.format(dims_umap.shape))
print('PCA reduced data E: {}'.format(dims_pca.shape))

<div class="alert-info">
5. Pregunta: Siendo $E^{(m,z)}$ el nuevo espacio generado por UMAP, donde $z=2$ y $E^{(m,n)}$ la dimension original. En el proceso de reduccion se cumple:
    <li> $E^{(m,n)} \rightarrow E^{(m,z)}$.</li>
    <li> $E^{(m,n)} \rightarrow E^{(m,n)}$.</li>
    <li> $E^{(n,m)} \rightarrow E^{(n,m)}$.</li>
    <li> $E^{(n,z)} \rightarrow E^{(n,z)}$.</li>
</div>

Como se puede observar, el numero de filas `m` se mantiene en todos los casos. Sin embargo, el numero de columnas para `UMAP` y `PCA` ha cambiado de $n \rightarrow 2$. Como tenemos $2$ dimensiones, procedamos a relizar algunas visualizaciones.

In [ ]:
plot_embeddings(dims_umap, dims_pca, targets)

Como habiamos comentado antes, `UMAP` es mas complejo que `PCA`, procedamos ahora a realizar una visualizacion mas compleja de las relaciones entre las distancias del espacio obtenido por `UMAP`.

In [ ]:
import umap.plot as plot

In [ ]:
plot.connectivity(umap, show_points=True, labels = targets);

<div class="alert-info">
6. Pregunta: Analizando los espacios obtenidos por UMAP y PCA, que diferencias significativas podria mencionar.
</div>

---

## 6. Transformaciones inversas en UMAP (de 2D a n):

Como mencionamos en la __seccion 5__ una vez que `UMAP` u `PCA` __aprenden__ a como representar $X^{(m,n)}$ en un __espacio__ $z$; es posible volver al __espacio original__ `n`, tal que: $X^{(m,z)} \rightarrow X^{(m,n)}$. A este proceso se le conoce formalmente como __reconstruccion__ o __transformacion inversa__.

En esta seccion vamos a realizar una __transformacion inversa__ con `UMAP`. Para ello, seleccionaremos un digito del dataset en la variable `n_class`. Recordemos que solo contamos con digitos en el intervalo $[0,9]$. Una vez completada la __seccion 7__, se le recomienda volver a esta seccion y probar con otros digitos.

In [ ]:
n_class = ...
sample, indx = get_sample(n_class, dims_umap, targets)

Ahora procedamos a inspeccionar el digito en la variable `sample`.

In [ ]:
sample

Como podemos observar contamos con un punto en 2D. Procedemos ahora a visualizar nuestra muestra en el espacio 2D de `UMAP`

In [ ]:
plot_sample(sample, dims_umap, targets)

Para realizar la operacion __inversa__, usaremos como entrada `sample` y generaremos las dimensiones originales en $E$. Para ello usaremos el comando `umap.inverse_transform(sample)`. Note que los resultados seran alamcenados en la variable `transform_sample`.

In [ ]:
# transform inverse UMAP
transform_sample = ...

Una vez transformado, procedamos a comprobar las dimensiones de la variable `transform_sample`:

In [ ]:
print('Umap sample 2D-points: {}'.format(sample.shape))
print('Umap sample inverse transform: {}'.format(transform_sample.shape))
print('Original encoded data E: {}'.format(encoded_data[indx, :].shape))

<div class="alert-info">
7. Pregunta: Siendo $s^{(m,z)}$ el vector sample, donde $z<n$ y $S$ el espacio original. En el proceso de transformacion inversa se cumple:
    <li> $s^{(m,z)} \rightarrow S^{(m,z)}$.</li>
    <li> $s^{(m,z)} \rightarrow S^{(m,n)}$.</li>
    <li> $s^{(m,z)} \rightarrow S^{(n,n)}$.</li>
    <li> $s^{(m,z)} \rightarrow S^{(m,z)}$.</li>
</div>

## 7. Generacion de numeros

Como hemos podido observar hemos sido capaces de convertir las dimensiones de $n \rightarrow 2$ y viceversa de $2 \rightarrow n$ usando `UMAP`. En esta seccion vamos a usar los datos de la seccion anterior para generar numeros usando el __decoder__. La idea para esta seccion consiste en usar la muestra en 2D de la variable `sample` para generar una imagen con el __decoder__. Sin embargo, recordemos que `sample` contiene solo $2$ dimensiones, mientras que el __decoder__ espera como entrada $n$. Es alli donde usaremos la __transformacion inversa__ de `UMAP` en `sample` para generar $2 \rightarrow n$. Esta transformacion se encuentra guardada en la variable `transform_sample` de la seccion anterior. La siguiente línea realizará el método que acabamos de describir:

In [ ]:
plot_transformation(sample, transform_sample, dims_umap, targets, n_class, auto_encoder)

Notemos que el numero que obtengamos esta definido por la variable `n_class`, si deseamos usar otro numero, tenemos que ejecutar el codigo desde la seccion anterior. Como se puede apreciar hemos generado un numero usando la reconstruccion inversa de `UMAP`. Procedamos ahora a comparar el numero generado usando los datos $E$ y los generados por `UMAP`.

In [ ]:
plot_comparison(encoded_data, transform_sample, auto_encoder, targets, indx)

Ahora, exploremos qué tipos de números puede generar nuestro __decodificador__. Para hacerlo, pasaremos todos los datos $E$ que contienen nuestro número seleccionado en la variable `n_class`. Usando estos datos vamos a generar un __subconjunto__ de muestras del número en `n_class`. Finalmente, usando la representacion 2D de `UMAP`, exploraremos en el espacio de UMAP, esto nos mostrará cómo el número en `n_class` puede mutar en diferentes números.

In [ ]:
plot_space(dims_umap, encoded_data, n_class, targets, auto_encoder, umap)

---